In [ ]:
def clean_text2(text):
    text = text.encode('utf-8').decode('unicode_escape')
    print("text after unicode_escape: ", test)
    text = text.replace(
        "\u201c", "“").replace(  # Left double quotation mark
        "\u201d", "”").replace(  # Right double quotation mark
        "\u2019", "’").replace(  # Right single quotation mark
        "\u2014", "—").replace(  # Em dash
        "\u2122", "™").replace(  # Trademark symbol
        "\u00a9", "©").replace(  # Copyright symbol
        "\u00ae", "®").replace(  # Registered trademark symbol
        "\u00f1", "ñ").replace(  # Lowercase n with tilde
        "\u00d1", "Ñ").replace(  # Uppercase N with tilde
        "\u00e9", "é").replace(  # Lowercase e with acute
        "\u00e8", "è").replace(  # Lowercase e with grave
        "\u00e2", "â").replace(  # Lowercase a with circumflex
        "\u00f3", "ó").replace(  # Lowercase o with acute
        "\u00f6", "ö").replace(  # Lowercase o with diaeresis
        "\u00f4", "ô").replace(  # Lowercase o with circumflex
        "\u00fc", "ü").replace(  # Lowercase u with diaeresis
        "\u00f7", "÷").replace(  # Division sign
        "\u00b0", "°").replace(  # Degree symbol
        "\u00a5", "¥").replace(  # Yen symbol
        "\u00b7", "·").replace(  # Middle dot
        "\u00e0", "à").replace(  # Lowercase a with grave
        "\u00df", "ß").replace(  # Eszett (sharp s)
        "\u2212", "−").replace(  # Minus sign
        "\u2b06", "⬆️").replace(  # Upwards button
        "\u2192", "→").replace(  # Rightwards arrow
        "\u2190", "←").replace(  # Leftwards arrow
        "\u2022", "•").replace(  # Bullet
        "\u2026", "…").replace(  # Ellipsis
        "\u2194", "↔️").replace(  # Left-right arrow
        "\u2500", "━").replace(  # Box drawings light horizontal
        "\ud83c\udf1f", "🌟").replace(  # Glowing star
        "\ud83d\ude80", "🚀")  # Rocket emoji

    print(text)


In [ ]:
# text to be cleaned
text1 = "ebm\u2011papst Invests \u20ac30 Million in New Site in Romania"
text2 = "Let\u2019s decarbonize district heating<br><br> together at this year\u2019s Euroheat &amp; Power Congress in Prague, Czech Republic, where"
text3 = 'Mark your calendars for May 27, 2025, at 11:00 CET, as we\\u2019re hosting an online session; Retrofitting Commercial Buildings: How E'



In [ ]:
clean_text2('Mark your calendars for May 27, 2025, at 11:00 CET, as we\\u2019re hosting an online session; Retrofitting Commercial Buildings: How E')

In [ ]:
# method 1 testing
clean_text2(text1)
print("\n")
clean_text2(text2)

In [ ]:
# method 2 testing
clean_text1(text1)
print("\n")
clean_text1(text2)

In [ ]:
import re
import html
from bs4 import BeautifulSoup

def clean_text1(text):
    if not text:
        return ""
    
    # 1. Unescape HTML entities (like &lt;, &gt;, etc.)
    text = html.unescape(text)
    
    # 2. Strip HTML tags using BeautifulSoup
    text = BeautifulSoup(text, "html.parser").get_text()

    # 3. Manually replace common Unicode characters for quotes and punctuation
    text = text.replace(
        "\u201c", "“").replace(  # Left double quotation mark
        "\u201d", "”").replace(  # Right double quotation mark
        "\u2019", "’").replace(  # Right single quotation mark
        "\u2014", "—").replace(  # Em dash
        "\u2122", "™").replace(  # Trademark symbol
        "\u00a9", "©").replace(  # Copyright symbol
        "\u00ae", "®").replace(  # Registered trademark symbol
        "\u00f1", "ñ").replace(  # Lowercase n with tilde
        "\u00d1", "Ñ").replace(  # Uppercase N with tilde
        "\u00e9", "é").replace(  # Lowercase e with acute
        "\u00e8", "è").replace(  # Lowercase e with grave
        "\u00e2", "â").replace(  # Lowercase a with circumflex
        "\u00f3", "ó").replace(  # Lowercase o with acute
        "\u00f6", "ö").replace(  # Lowercase o with diaeresis
        "\u00f4", "ô").replace(  # Lowercase o with circumflex
        "\u00fc", "ü").replace(  # Lowercase u with diaeresis
        "\u00f7", "÷").replace(  # Division sign
        "\u00b0", "°").replace(  # Degree symbol
        "\u00a5", "¥").replace(  # Yen symbol
        "\u00b7", "·").replace(  # Middle dot
        "\u00e0", "à").replace(  # Lowercase a with grave
        "\u00df", "ß").replace(  # Eszett (sharp s)
        "\u2212", "−").replace(  # Minus sign
        "\u2b06", "⬆️").replace(  # Upwards button
        "\u2192", "→").replace(  # Rightwards arrow
        "\u2190", "←").replace(  # Leftwards arrow
        "\u2022", "•").replace(  # Bullet
        "\u2026", "…").replace(  # Ellipsis
        "\u2194", "↔️").replace(  # Left-right arrow
        "\u2500", "━").replace(  # Box drawings light horizontal
        "\ud83c\udf1f", "🌟").replace(  # Glowing star
        "\ud83d\ude80", "🚀")  # Rocket emoji

    # 4. Remove any invalid surrogate characters (if any)
    text = re.sub(r'[\ud800-\udfff]', '', text)

    # 5. Remove extra spaces, tabs, and newlines
    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("articles.csv")

# Display the first few rows of the DataFrame
df.head()

In [ ]:
# Apply the clean_text1 method to the "title" and "summary" columns

test = pd.DataFrame()

test['cleaned_title'] = df['title'].apply(lambda x: clean_text1(str(x)) if pd.notnull(x) else "")
test['cleaned_summary'] = df['summary'].apply(lambda x: clean_text1(str(x)) if pd.notnull(x) else "")

# Display the first few rows of the updated DataFrame

# Post test: Filter records where '\u2019' is present in either the title or summary column
filtered_records3 = test[test['cleaned_title'].str.contains(r"\\u2019", na=False) | test['cleaned_summary'].str.contains(r"\\u2019", na=False)]

# Post test: Display the filtered records
filtered_records3['cleaned_title'].iloc[0]

In [ ]:
import re

def clean_text_manually(text):
    if not isinstance(text, str):
        return ""

    text = text.replace("&amp;", "&").replace("&lt;", "<").replace("&gt;", ">") \
               .replace("&quot;", '"').replace("&apos;", "'").replace("&nbsp;", " ") \
               .replace("&copy;", "©").replace("&reg;", "®").replace("&eacute;", "é") \
               .replace("&mdash;", "—").replace("&ndash;", "–").replace("&hellip;", "…") \
               .replace("&lsquo;", "‘").replace("&rsquo;", "’").replace("&ldquo;", "“") \
               .replace("&rdquo;", "”") \
               .replace("\u201c", "“").replace("\u201d", "”").replace("\u2019", "’") \
               .replace("\u2014", "—").replace("\u2122", "™").replace("\u00a9", "©") \
               .replace("\u00ae", "®").replace("\u00f1", "ñ").replace("\u00d1", "Ñ") \
               .replace("\u00e9", "é").replace("\u00e8", "è").replace("\u00e2", "â") \
               .replace("\u00f3", "ó").replace("\u00f6", "ö").replace("\u00f4", "ô") \
               .replace("\u00fc", "ü").replace("\u00f7", "÷").replace("\u00b0", "°") \
               .replace("\u00a5", "¥").replace("\u00b7", "·").replace("\u00e0", "à") \
               .replace("\u00df", "ß").replace("\u2212", "−").replace("\u2b06", "⬆️") \
               .replace("\u2192", "→").replace("\u2190", "←").replace("\u2022", "•") \
               .replace("\u2026", "…").replace("\u2194", "↔️").replace("\u2500", "━") \
               .replace("\ud83c\udf1f", "🌟").replace("\ud83d\ude80", "🚀")

    # Remove HTML tags
    text = re.sub(r"<[^>]+>", "", text)

    return text.strip()


In [ ]:
df['title'] = df['title'].apply(lambda x: clean_text_manually(str(x)) if pd.notnull(x) else "")
df['summary'] = df['summary'].apply(lambda x: clean_text_manually(str(x)) if pd.notnull(x) else "")

# Post test: Filter records where '\u2019' is present in either the title or summary column
filtered_records2 = df[df['title'].str.contains(r"\\u2019", na=False) | df['summary'].str.contains(r"\\u2019", na=False)]

# Post test: Display the filtered records
filtered_records2

In [ ]:
# Post test: Filter records where '\u2019' is present in either the title or summary column
filtered_records1 = df[df['title'].str.contains(r"\\u2019", na=False) | df['summary'].str.contains(r"\\u2019", na=False)]

# Post test: Display the filtered records
filtered_records1

In [ ]:
# Filter records where '\u2019' is present in either the title or summary column
filtered_records = df[df['title'].str.contains(r"\\u2019", na=False) | df['summary'].str.contains(r"\\u2019", na=False)]

# Display the filtered records
filtered_records